In [ ]:
! pip install -U sentence-transformers
! pip install -q wandb py_vncorenlp

import py_vncorenlp
py_vncorenlp.download_model(save_dir='/kaggle/working/')
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/default-java'
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/working')

In [2]:
import pandas as pd
import numpy as np
import torch
import pickle
import math
import json
from datasets import load_dataset, DatasetDict, Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, losses, util, datasets
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
import numpy as np

2024-06-12 09:17:07.752748: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 09:17:07.752865: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 09:17:07.896622: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
no_segment_flatten_corpus = []
with open("/kaggle/input/law-qa/flatten_law_corpus.json", "r") as file:
    for line in file:
        data = json.loads(line)
        no_segment_flatten_corpus.append(data)

In [4]:
no_segment_flatten_corpus[0]

{'article_id': '32/2024/qh15_1',
 'name': 'Điều 1. Phạm vi điều chỉnh',
 'content': ['Luật này quy định về việc thành lập, tổ chức, hoạt động, can thiệp sớm, kiểm soát đặc biệt, tổ chức lại, giải thể, phá sản tổ chức tín dụng; việc thành lập, tổ chức, hoạt động, can thiệp sớm, giải thể, chấm dứt hoạt động của chi nhánh ngân hàng nước ngoài; việc thành lập, hoạt động của văn phòng đại diện tại Việt Nam của tổ chức tín dụng nước ngoài, tổ chức nước ngoài khác có hoạt động ngân hàng; việc xử lý nợ xấu, tài sản bảo đảm của khoản nợ xấu của tổ chức tín dụng, chi nhánh ngân hàng nước ngoài, tổ chức mà Nhà nước sở hữu 100% vốn điều lệ có chức năng mua, bán, xử lý nợ.']}

In [5]:
no_segment_queries = []

with open("/kaggle/input/law-qa/query_set_evaluate.json", "r") as file:
    for line in file:
        data = json.loads(line)
        no_segment_queries.append(data)

In [6]:
no_segment_queries[3]

{'query_id': '66635717c2f544363eedc008',
 'query': '03 hình thức xuất khẩu lao động theo hợp đồng hợp pháp hiện nay là gì?',
 'relevant_docs': ['69/2020/qh14_5']}

In [7]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(no_segment_queries, test_size=0.2, random_state=42)

In [8]:
print("Length train set:", len(train_set))
print("Length test set:", len(test_set))

Length train set: 14872
Length test set: 3719


In [9]:
no_segment_flatten_queries_train = []
for query in train_set:
    for rd in query['relevant_docs']:
        no_segment_flatten_queries_train.append({
            'query_id': query['query_id'],
            'query': query['query'],
            'relevant_doc': rd,
        })

In [10]:
no_segment_flatten_queries_train[3]

{'query_id': '666357a3c2f544363ef06c08',
 'query': 'Mua bán nhà đất bằng vàng thì đóng thuế thu nhập cá nhân bằng cách nào?',
 'relevant_doc': '04/2007/qh12_6'}

In [11]:
def process_relevant_doc_id(relevant_doc_id):
    second_underscore_index = relevant_doc_id.find("_", relevant_doc_id.find("_") + 1)
    if second_underscore_index != -1:
        id_part = relevant_doc_id[:second_underscore_index]
        chunk_part = relevant_doc_id[second_underscore_index + 1:]
        chunk_part = int(chunk_part) if chunk_part.isdigit() else None
        return id_part, chunk_part
    else:
        return relevant_doc_id, None

In [12]:
import re
def chunk_text_with_subheadings(text):
    if re.search(r'\d+\.', text) or re.search(r'[a-z]\)', text, flags=re.IGNORECASE):
        chunks = re.split(r'\n(?=\d+\.)', text)
        final_chunks = [chunk.strip() for chunk in chunks]
    else:
        final_chunks = [text.strip()]
    
    return final_chunks

In [13]:
def get_first_chunk_with_part(list_chunks, chunk_part):
    pattern = re.compile(rf'^\s*{chunk_part}\s*[\.\)\s]')
    
    for chunk in list_chunks:
        if pattern.match(chunk):
            return chunk
    
    return None

In [14]:
no_segment_data_train = []
corpus_dict = {doc['article_id']: doc for doc in no_segment_flatten_corpus}

for query in no_segment_flatten_queries_train:
    relevant_doc_id, chunk_part = process_relevant_doc_id(query['relevant_doc'])
    
    if relevant_doc_id in corpus_dict:
        matched_doc = corpus_dict[relevant_doc_id]
        if chunk_part is None:
            content_str = ' '.join(map(str, matched_doc['content']))
            positive = f"{matched_doc['name']}. {content_str}"
            no_segment_data_train.append({
                'query': query['query'],
                'positive': positive
            })
        else:
            list_chunks = chunk_text_with_subheadings(' '.join(map(str, matched_doc['content'])))
            chunk_part_match = get_first_chunk_with_part(list_chunks, chunk_part )
            if chunk_part_match is not None:
                no_segment_data_train.append({
                'query': query['query'],
                'positive': f"{matched_doc['name']}. {chunk_part_match}"
            })
            

In [15]:
no_segment_data_train[0]

{'query': 'Lao động nữ cấy que tránh thai có được hưởng chế độ thai sản không?',
 'positive': 'Điều 31. Điều kiện hưởng chế độ thai sản. 1. Người lao động được hưởng chế độ thai sản khi thuộc một trong các trường hợp sau đây: a) Lao động nữ mang thai; b) Lao động nữ sinh con; c) Lao động nữ mang thai hộ và người mẹ nhờ mang thai hộ; d) Người lao động nhận nuôi con nuôi dưới 06 tháng tuổi; đ) Lao động nữ đặt vòng tránh thai, người lao động thực hiện biện pháp triệt sản; e) Lao động nam đang đóng bảo hiểm xã hội có vợ sinh con. 2. Người lao động quy định tại các điểm b, c và d khoản 1 Điều này phải đóng bảo hiểm xã hội từ đủ 06 tháng trở lên trong thời gian 12 tháng trước khi sinh con hoặc nhận nuôi con nuôi. 3. Người lao động quy định tại điểm b khoản 1 Điều này đã đóng bảo hiểm xã hội từ đủ 12 tháng trở lên mà khi mang thai phải nghỉ việc để dưỡng thai theo chỉ định của cơ sở khám bệnh, chữa bệnh có thẩm quyền thì phải đóng bảo hiểm xã hội từ đủ 03 tháng trở lên trong thời g

In [16]:
segmented_data_train = []
for query in no_segment_data_train:
    segmented_data_train.append({
        'query' : ' '.join(rdrsegmenter.word_segment(query['query'])),
        'positive' : ' '.join(rdrsegmenter.word_segment(query['positive']))
    })

In [17]:
print(segmented_data_train[0])

{'query': 'Lao_động nữ cấy que tránh thai có được hưởng chế_độ thai_sản không ?', 'positive': 'Điều 31 . Điều_kiện hưởng chế_độ thai_sản . 1 . Người lao_động được hưởng chế_độ thai_sản khi thuộc một trong các trường_hợp sau đây : a ) Lao_động nữ mang thai ; b ) Lao_động nữ_sinh con ; c ) Lao_động nữ mang thai hộ và người mẹ nhờ mang thai hộ ; d ) Người lao_động nhận nuôi con_nuôi dưới 06 tháng tuổi ; đ ) Lao_động nữ đặt_vòng tránh thai , người lao_động thực_hiện biện_pháp triệt_sản ; e ) Lao_động nam đang đóng bảo_hiểm_xã_hội có vợ sinh con . 2 . Người lao_động quy_định tại các điểm b , c và d khoản 1 Điều này phải đóng bảo_hiểm_xã_hội từ đủ 06 tháng trở lên trong thời_gian 12 tháng trước khi sinh con hoặc nhận nuôi con_nuôi . 3 . Người lao_động quy_định tại điểm b khoản 1 Điều này đã đóng bảo_hiểm_xã_hội từ đủ 12 tháng trở lên mà khi mang thai phải nghỉ việc để dưỡng thai theo chỉ định của cơ_sở khám bệnh , chữa bệnh có thẩm quyền thì phải đóng bảo_hiểm_xã_hội từ đủ 03 thán

In [18]:
train_dataset = []
train_dataset = [InputExample(texts = [a['query'], a['positive']]) for a in segmented_data_train]

In [19]:
word_embedding_model = models.Transformer("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode = 'mean')

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [20]:
num_epochs = 5
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
train_loss = losses.MultipleNegativesRankingLoss(model=model)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)

In [21]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    optimizer_class=torch.optim.AdamW,
    optimizer_params={
        'lr': 2e-5
    },
    weight_decay=0.01,
    output_path="/kaggle/working/finetuned_simcse_5epoch_15k"
)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.136000
1000,0.033600
1500,0.021000


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

# Evaluate

In [22]:
import json
import pickle

with open('/kaggle/input/law-train-evalute/chunk_id_mapping', 'rb') as f:
    idx_mapping = pickle.load(f)
with open('/kaggle/input/law-train-evalute/chunk_corpus', 'rb') as f:
    segmented_chunk_corpus = pickle.load(f)
with open('/kaggle/input/law-train-evalute/queries_relevant_chunks', 'rb') as f:
    segmented_queries_relevant_chunk = pickle.load(f)
    
query_test_set = test_set

In [23]:
print(segmented_chunk_corpus[10])

{'doc_id': '32/2024/qh15_4', 'chunk_id': '32/2024/qh15_4_3', 'title': 'Điều 4 .Giải_thích từ_ngữ', 'chunk': '3 . Can_thiệp sớm là việc Ngân_hàng Nhà_nước Việt_Nam ( sau đây gọi là Ngân_hàng Nhà_nước ) áp_dụng các yêu_cầu , biện_pháp hạn_chế đối_với tổ_chức tín_dụng , chi_nhánh ngân_hàng nước_ngoài và yêu_cầu tổ_chức tín_dụng , chi_nhánh ngân_hàng nước_ngoài đó thực_hiện phương_án khắc_phục dưới sự giám_sát của Ngân_hàng Nhà_nước nhằm khắc_phục tình_trạng theo quy_định tại khoản 1 Điều 156 của Luật này .', 'text': 'Điều 4 .Giải_thích từ_ngữ :3 . Can_thiệp sớm là việc Ngân_hàng Nhà_nước Việt_Nam ( sau đây gọi là Ngân_hàng Nhà_nước ) áp_dụng các yêu_cầu , biện_pháp hạn_chế đối_với tổ_chức tín_dụng , chi_nhánh ngân_hàng nước_ngoài và yêu_cầu tổ_chức tín_dụng , chi_nhánh ngân_hàng nước_ngoài đó thực_hiện phương_án khắc_phục dưới sự giám_sát của Ngân_hàng Nhà_nước nhằm khắc_phục tình_trạng theo quy_định tại khoản 1 Điều 156 của Luật này .'}


In [24]:
print(query_test_set[10])

{'query_id': '66635637c2f544363eeb0fef', 'query': 'Công ty đại chúng đang có nợ phải trả quá hạn thì có được mua lại cổ phiếu của chính mình không?', 'relevant_docs': ['54/2019/qh14_36_3']}


In [25]:
query_test_relevant_chunks = []

for query in query_test_set:
    
    relevant_chunks = []
    
    for relevant_doc in query['relevant_docs']:
        try:
            relevant_chunks += idx_mapping[relevant_doc]
        except KeyError:
            continue
        
    query_test_relevant_chunks.append({
        'query_id' : query['query_id'],
        'query' : query['query'],
        'relevant_docs' : query['relevant_docs'],
        'relevant_chunks' : relevant_chunks
    })

In [26]:
segmented_query_test_relevant_chunks = []

for query in query_test_relevant_chunks:
    segmented_query_test_relevant_chunks.append({
        'query_id' : query['query_id'],
        'query' : ''.join(rdrsegmenter.word_segment(query['query'])),
        'relevant_docs' : query['relevant_docs'],
        'relevant_chunks' : query['relevant_chunks']
    })

In [27]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = {}
queries = {}
relevant_docs = {}
corpus_idx = 0
query_idx = 0

for query in segmented_query_test_relevant_chunks:
    queries[query['query_id']] = query['query']
    relevant_docs[query['query_id']] = set(query['relevant_chunks'])

for doc in segmented_chunk_corpus:
    corpus[doc['chunk_id']] = doc['text']

## Evalute 15k - 5 epochs

In [28]:
IR_evaluator = InformationRetrievalEvaluator(queries,corpus,relevant_docs)
model.evaluate(IR_evaluator, output_path="/kaggle/working/finetuned_simcse_5epoch_15k_chunk")

{'cosine_accuracy@1': 0.5232606010703993,
 'cosine_accuracy@3': 0.7299300123507616,
 'cosine_accuracy@5': 0.7941539728283245,
 'cosine_accuracy@10': 0.868670234664471,
 'cosine_precision@1': 0.5232606010703993,
 'cosine_precision@3': 0.4158089748867847,
 'cosine_precision@5': 0.34318649650061755,
 'cosine_precision@10': 0.22737752161383284,
 'cosine_recall@1': 0.17797812551414854,
 'cosine_recall@3': 0.3989706860847248,
 'cosine_recall@5': 0.5224093473959673,
 'cosine_recall@10': 0.6587583750665266,
 'cosine_ndcg@10': 0.5714609186617473,
 'cosine_mrr@10': 0.6424728969397552,
 'cosine_map@100': 0.5033450756346438,
 'dot_accuracy@1': 0.5249073692877727,
 'dot_accuracy@3': 0.7262247838616714,
 'dot_accuracy@5': 0.7879785920131741,
 'dot_accuracy@10': 0.8583779333058872,
 'dot_precision@1': 0.5249073692877727,
 'dot_precision@3': 0.41210374639769454,
 'dot_precision@5': 0.3364347468093866,
 'dot_precision@10': 0.22400164676821735,
 'dot_recall@1': 0.17735941870976862,
 'dot_recall@3': 0.39

In [29]:
import pandas as pd
df = pd.read_csv('/kaggle/working/finetuned_simcse_5epoch_15k_chunk/Information-Retrieval_evaluation_results.csv')
df

,epoch,steps,cosine-Accuracy@1,cosine-Accuracy@3,cosine-Accuracy@5,cosine-Accuracy@10,cosine-Precision@1,cosine-Recall@1,cosine-Precision@3,cosine-Recall@3,...,dot-Recall@1,dot-Precision@3,dot-Recall@3,dot-Precision@5,dot-Recall@5,dot-Precision@10,dot-Recall@10,dot-MRR@10,dot-NDCG@10,dot-MAP@100
0,-1,-1,0.523261,0.72993,0.794154,0.86867,0.523261,0.177978,0.415809,0.398971,...,0.177359,0.412104,0.396072,0.336435,0.512217,0.224002,0.650004,0.639589,0.564935,0.497805
